<a href="https://colab.research.google.com/github/dkanh6/Computational_Neuroscience/blob/Custom/HCO_Single_Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Single Cell Spiking Model**

This script will model a single compartment neuron that will have classic AP spiking activity.


## **Notebook Setup**
---

### **Install Neuron**
first install NEURON package, import .h submodule, and matplotlib + numpy


In [13]:
!pip install neuron
from neuron import h
import matplotlib.pyplot as plt
import numpy as np
h.load_file('stdrun.hoc')

1.0

### **Download modfiles from hithub**


In [14]:
import os
!git clone https://github.com/chenziao/HCO-Model-Project-Colab.git
%cd HCO-Model-Project-Colab/


Cloning into 'HCO-Model-Project-Colab'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 43 (delta 20), reused 5 (delta 3), pack-reused 0
Receiving objects: 100% (43/43), 15.47 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/HCO-Model-Project-Colab/HCO-Model-Project-Colab/HCO-Model-Project-Colab


### **Compile modfiles**

In [15]:
flag = os.system('nrnivmodl') # compiles modfiles. Returns 0 for success, 1 for failure
if(flag==0):
  print("Mod Files Compiled Successfully!")
else:
  print("Mod Failes Compilation Failed")

Mod Files Compiled Successfully!


## **Model Setup**
This will be a simple single compartment cell model


### **Initalize the Model**


In [22]:
from HCO_cell_template import HCOcell

# Create a single Cell
cell = HCOcell()
print("Here is the length of the cell " + str(cell.soma.L))

# Create a current Clamp

# Create a synaptic Input to the Cell
h.dt = 0.025 # time step (resolution) of the simulation in ms
h.tstop = 200 # how long the smiulation should run in ms



Here is the length of the cell 1000.0
